In [15]:
pip install yfinance

In [17]:
pip install --upgrade pandas_datareader

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import yfinance as yf
import statsmodels.api as sm

In [42]:
# Define the stock symbol and date range
symbol = 'AAPL'
start_date = '2020-01-01'
end_date = '2022-01-01'

# Fetch historical data using yfinance
stock_data = yf.download(symbol, start=start_date, end=end_date)

# Print the obtained data
print(stock_data)

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-02   74.059998   75.150002   73.797501   75.087502   73.152657   
2020-01-03   74.287498   75.144997   74.125000   74.357498   72.441460   
2020-01-06   73.447502   74.989998   73.187500   74.949997   73.018677   
2020-01-07   74.959999   75.224998   74.370003   74.597504   72.675278   
2020-01-08   74.290001   76.110001   74.290001   75.797501   73.844337   
...                ...         ...         ...         ...         ...   
2021-12-27  177.089996  180.419998  177.070007  180.330002  178.292877   
2021-12-28  180.160004  181.330002  178.529999  179.289993  177.264618   
2021-12-29  179.330002  180.630005  178.139999  179.380005  177.353607   
2021-12-30  179.470001  180.570007  178.089996  178.199997  176.186935   
2021-12-31  178.089996  179.229996  177.259995  177.570007  175.564041   

               Volume  
Date         

In [43]:
# Export data to Excel
excel_filename = 'stock_data.xlsx'
stock_data.to_excel(excel_filename)
print(f'Data exported to {excel_filename}')

Data exported to stock_data.xlsx


# **Calculating Alpha, Beta and Sharpe Ratios**

In [82]:
# Calculate daily returns
stock_data['Daily Returns'] = stock_data['Adj Close'].pct_change()

# Fetch S&P 500 data for market returns
market_data = yf.download('^GSPC', start=start_date, end=end_date)
market_data['Daily Returns'] = market_data['Adj Close'].pct_change()

# Calculate market return
market_return = market_data['Daily Returns'].mean()

# Merge stock and market data
merged_data = pd.merge(stock_data[['Daily Returns']], market_data[['Daily Returns']], left_index=True, right_index=True, suffixes=('_stock', '_market'))

# Drop missing values
merged_data = merged_data.dropna()

# Extract dependent and independent variables
y = merged_data['Daily Returns_stock']
X = sm.add_constant(merged_data['Daily Returns_market'])

# Calculate alpha and beta using linear regression
model = sm.OLS(y, X).fit()
alpha, beta = model.params['const'], model.params['Daily Returns_market']

# Calculate Sharpe ratio
risk_free_rate = 0.01
sharpe_ratio = (np.mean(y - risk_free_rate) / np.std(y))

# Print the results
print(f'Market Return: {market_return:.6f}')
print(f'Alpha: {alpha:.6f}')
print(f'Beta: {beta:.6f}')
print(f'Sharpe Ratio: {sharpe_ratio:.6f}')


[*********************100%%**********************]  1 of 1 completed

Market Return: 0.001057
Alpha: 0.000045
Beta: 1.309363
Sharpe Ratio: -0.544000


# **CAPM Calculation**

In [61]:
# Variables required for CAPM calculation
risk_free_rate = 0.01
market_return = 0.001057
beta = 1.309363

# Convert the risk-free rate, market return, and beta to decimals
risk_free_rate /= 100
market_return /= 100

# Display the converted values
print(f"Converted Risk-Free Rate: {risk_free_rate}")
print(f"Converted Market Return: {market_return}")

Converted Risk-Free Rate: 0.0001
Converted Market Return: 1.057e-05


In [83]:
# Variables required for CAPM calculation
risk_free_rate = 0.01
market_return = 0.001057
beta = 1.309363

# Convert the decimal values to percentages
risk_free_rate_percentage = risk_free_rate * 100
market_return_percentage = market_return * 100
beta_percentage= beta * 100

# Display the converted values
print(f"Converted Risk-Free Rate: {risk_free_rate_percentage}%")
print(f"Converted Market Return: {market_return_percentage}%")
print(f"Converted Beta: {beta_percentage}%")

Converted Risk-Free Rate: 1.0%
Converted Market Return: 0.1057%
Converted Beta: 130.93630000000002%


In [ ]:
def calculate_expected_return(risk_free_rate, market_return, beta):
    expected_return = risk_free_rate + beta * (market_return - risk_free_rate)
    return expected_return

def main():
    risk_free_rate, market_return, beta = get_user_input()
    expected_return = calculate_expected_return(risk_free_rate, market_return, beta)
    display_results(expected_return)

def get_user_input():
    risk_free_rate = float(input("Enter the risk-free rate (%): ")) / 100
    market_return = float(input("Enter the market return (%): ")) / 100
    beta = float(input("Enter the beta: "))
    return risk_free_rate, market_return, beta

def display_results(expected_return):
    print(f"\nExpected Return: {expected_return * 100:.2f}%")

def main():
    risk_free_rate, market_return, beta = get_user_input()
    expected_return = calculate_expected_return(risk_free_rate, market_return, beta)
    display_results(expected_return)

if __name__ == "__main__":
    main()


# **User Interface - Interactive Widgets**

In [79]:
from ipywidgets import interact, widgets
import pandas as pd

def capm_calculator(risk_free_rate, market_return, beta):
    expected_return = risk_free_rate + beta * (market_return - risk_free_rate)
    return expected_return

# Interactive user interface using widgets
@interact
def calculate_capm(expected_risk_free_rate=widgets.FloatSlider(min=0, max=0.1, step=0.01, value=0.02, description='Risk-Free Rate'),
                   expected_market_return=widgets.FloatSlider(min=0, max=0.2, step=0.01, value=0.08, description='Market Return'),
                   expected_beta=widgets.FloatSlider(min=0, max=2, step=0.1, value=1.0, description='Beta')):

    # Perform CAPM calculation
    result = capm_calculator(expected_risk_free_rate, expected_market_return, expected_beta)

    # Display result
    print(f"Expected Return: {result * 100:.2f}%")


interactive(children=(FloatSlider(value=0.02, description='Risk-Free Rate', max=0.1, step=0.01), FloatSlider(v…

# **User Interface - HTML Page**

In [80]:
# HTML code for the user interface
html_code = '''
<!DOCTYPE html>
<html>
<head>
  <title>CAPM Calculator</title>
  <script>
    function calculateCAPM() {
      var riskFreeRate = parseFloat(document.getElementById('riskFreeRate').value) / 100;
      var marketReturn = parseFloat(document.getElementById('marketReturn').value) / 100;
      var beta = parseFloat(document.getElementById('beta').value);

      // CAPM Calculation
      var expectedReturn = riskFreeRate + beta * (marketReturn - riskFreeRate);

      // Display the result
      document.getElementById('result').innerHTML = 'Expected Return: ' + (expectedReturn * 100).toFixed(2) + '%';
    }
  </script>
</head>
<body>
  <h2>CAPM Calculator</h2>
  <form>
    <label for="riskFreeRate">Risk-Free Rate (%): </label>
    <input type="text" id="riskFreeRate"><br>

    <label for="marketReturn">Market Return (%): </label>
    <input type="text" id="marketReturn"><br>

    <label for="beta">Beta: </label>
    <input type="text" id="beta"><br>

    <button type="button" onclick="calculateCAPM()">Calculate</button>
  </form>

  <div id="result"></div>
</body>
</html>
'''

# Save HTML code to an HTML file
with open('capm_calculator.html', 'w') as file:
    file.write(html_code)

In [81]:
from IPython.display import HTML

# Display the HTML file
HTML(filename='capm_calculator.html')